In [2]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [3]:
import os
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [4]:
llm=ChatGroq(model="openai/gpt-oss-120b")

In [5]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [6]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}



In [7]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [8]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)


{'topic': 'pizza',
 'joke': 'Why did the pizza apply for a job?  \n\nBecause it heard the company was looking for someone with *extra* cheese and a great *crust*‑om! 🍕😄',
 'explanation': '**Explanation of the joke**\n\n| Part of the joke | Why it’s funny |\n|------------------|----------------|\n| **“Why did the pizza apply for a job?”** | Sets up a classic “knock‑knock” or “why did the …” format, where the answer will involve a pun. The absurd image of a pizza—an inanimate food item—trying to get a job is already a bit silly, priming the listener for wordplay. |\n| **“Because it heard the company was looking for someone with *extra* cheese…”** | *Extra cheese* is a common pizza topping request (people love more cheese). In a corporate context, “extra cheese” sounds like “extra *cheese*” = “extra *cash*” or “extra *flair*,” suggesting the company wants someone who brings added value or “cheesy” charisma. The word “cheese” also evokes the idiom “cheesy” meaning overly sentimental or cor

In [27]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the pizza apply for a job?  \n\nBecause it heard the company was looking for someone with *extra* cheese and a great *crust*‑om! 🍕😄'}, next=('generate_explanation',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10198b-e9ca-67fa-8002-09427645fb53'}}, metadata={'source': 'update', 'step': 2, 'parents': {}}, created_at='2026-02-04T07:11:36.979003+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f101932-340e-66c0-8001-d36ed3bf94f2'}}, tasks=(PregelTask(id='dfc4b414-7f32-fa17-542f-d4660ae3bb5d', name='generate_explanation', path=('__pregel_pull', 'generate_explanation'), error=None, interrupts=(), state=None, result=None),), interrupts=())

In [28]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the pizza apply for a job?  \n\nBecause it heard the company was looking for someone with *extra* cheese and a great *crust*‑om! 🍕😄'}, next=('generate_explanation',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10198b-e9ca-67fa-8002-09427645fb53'}}, metadata={'source': 'update', 'step': 2, 'parents': {}}, created_at='2026-02-04T07:11:36.979003+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f101932-340e-66c0-8001-d36ed3bf94f2'}}, tasks=(PregelTask(id='dfc4b414-7f32-fa17-542f-d4660ae3bb5d', name='generate_explanation', path=('__pregel_pull', 'generate_explanation'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the pizza apply for a job?  \n\nBecause it heard the company was looking for someone with *extra* cheese and a great *crust*‑om! 🍕😄'}, next=('generat

In [12]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'ant'}, config=config2)

{'topic': 'ant',
 'joke': 'Why did the ant bring a suitcase to the picnic?\n\nBecause it heard the food was *un‑ant‑icipated* and wanted to be ready for a **big bite**! 🐜😄',
 'explanation': '**What makes the joke funny?**  \n\n| Part of the joke | Why it’s a pun / wordplay | How it contributes to the humor |\n|------------------|--------------------------|--------------------------------|\n| **“Why did the ant bring a suitcase to the picnic?”** | Sets up an absurd visual: an ant (a creature that’s only a few millimetres long) trying to pack a *suitcase*. The mental image is already funny because a suitcase is huge compared to an ant. | The absurdity primes the listener for a punch‑line that will involve a play on words. |\n| **“Because it heard the food was *un‑ant‑icipated*…”** | The word *un‑anticipated* is split into **“un‑ant‑icipated.”** By inserting the word **ant** into the middle of *anticipated*, the joke creates a new, nonsense adjective that sounds just like the real word. T

###TIME TRAVEL

In [25]:
workflow.get_state({'configurable': {'thread_id': '1', 'checkpoint_id': '1f10198b-e9ca-67fa-8002-09427645fb53'}})

StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the pizza apply for a job?  \n\nBecause it heard the company was looking for someone with *extra* cheese and a great *crust*‑om! 🍕😄'}, next=('generate_explanation',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f10198b-e9ca-67fa-8002-09427645fb53'}}, metadata={'source': 'update', 'step': 2, 'parents': {}}, created_at='2026-02-04T07:11:36.979003+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f101932-340e-66c0-8001-d36ed3bf94f2'}}, tasks=(PregelTask(id='dfc4b414-7f32-fa17-542f-d4660ae3bb5d', name='generate_explanation', path=('__pregel_pull', 'generate_explanation'), error=None, interrupts=(), state=None, result=None),), interrupts=())

In [26]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the pizza apply for a job?  \n\nBecause it heard the company was looking for someone with *extra* cheese and a great *crust*‑om! 🍕😄'}, next=('generate_explanation',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f10198b-e9ca-67fa-8002-09427645fb53'}}, metadata={'source': 'update', 'step': 2, 'parents': {}}, created_at='2026-02-04T07:11:36.979003+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f101932-340e-66c0-8001-d36ed3bf94f2'}}, tasks=(PregelTask(id='dfc4b414-7f32-fa17-542f-d4660ae3bb5d', name='generate_explanation', path=('__pregel_pull', 'generate_explanation'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the pizza apply for a job?  \n\nBecause it heard the company was looking for someone with *extra* cheese and a great *crust*‑om! 🍕😄'}, next=('generat

In [23]:
workflow.update_state({'configurable': {'thread_id': '1', 'checkpoint_id': '1f101932-340e-66c0-8001-d36ed3bf94f2',  "checkpoint_ns": ""}}, {'topic':'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f10198b-e9ca-67fa-8002-09427645fb53'}}